In [201]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
# bc = BertClient(ip='localhost', check_version=False, check_length=False)
class TrainingQuery(Dataset):
    def __init__(self, raw_training_data, news_urls, contents):
        # read provided data
        # proccess data
        merged_training = pd.merge(raw_training_data, news_urls, on=['News_Index'])

        # construct dataset
        self.queries = merged_training['Query']
        self.contents = contents[merged_training['News_URL']]
        self.targets = merged_training['Relevance']
        
        self.contents = list(self.contents)
        self.queries = list(self.queries)
    
        self.size = len(merged_training)
        
    def __getitem__(self, i):
        return self.queries[i], self.contents[i], self.targets[i]
    
    def __len__(self):
        return self.size


In [3]:
class ARGS():
    lr = 0.0001
    batch = 50
    epoch = 100
    num_layers = 2
    seq_len = 30
    word_dim =  768
    hidden_dim = 200

In [302]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)
# train_dataset = TrainingQuery(raw_training_data, news_urls, contents)
# train_data, test_data = train_test_split(raw_training_data, test_size=0.1)
# train_dataset = TrainingQuery(train_data, news_urls, contents)
# test_dataset = TrainingQuery(test_data, news_urls, contents)

In [136]:
content_list = list(contents)

In [158]:
all_doc = contents # (train_dataset.contents)
sentenece_arr = [ jieba.lcut(sen) for sen in all_doc]

In [159]:
from gensim import corpora
word_list = sentenece_arr
dictionary = corpora.Dictionary(word_list)
new_corpus = [ dictionary.doc2bow(text) for text in word_list]

In [161]:
doc_dict = {}
for idx, text in enumerate(new_corpus):
    all_words = ([i[0] for i in text])
    
    for word in all_words:
        if word in doc_dict.keys():
            doc_dict[word].append(idx)
        else:
            doc_dict[word]= [idx]

In [163]:
len(doc_dict), len(sentenece_arr)

(1201423, 100000)

In [242]:
total_scores = list()

for test_id , text_q in enumerate(test_query):
    print( test_id ,text_q)
    text = jieba.lcut(text_q)
    text_bow = dictionary.doc2bow(text) 
    keys, values = zip(*tfidf[text_bow])
    
    for cnt, key in enumerate(keys):
        if cnt == 0:
            queried_doc = set(doc_dict[key])
        else:
            queried_doc = set(doc_dict[key]) | queried_doc
    
    cosine_sim_score = list()
    for cnt, doc in enumerate(queried_doc):
        tfidf_score = np.zeros((len(keys)))
        result = sentenece_arr[doc]
        result = dictionary.doc2bow(result)
        result_dict = dict(tfidf[result])
        
        for idx, i in enumerate(keys):
            if i in result_dict.keys():
                tfidf_score[idx] = result_dict[i]
        
        tfidf_score, values  = np.array(tfidf_score).reshape(1,-1), np.array(values).reshape(1,-1)
        score = cosine_similarity(tfidf_score, values)
        cosine_sim_score.append((doc, score[0][0]))
        
    total_scores.append(cosine_sim_score)

0 通姦在刑法上應該除罪化
1 應該取消機車強制二段式左轉(待轉)
2 支持博弈特區在台灣合法化
3 中華航空空服員罷工是合理的
4 性交易應該合法化
5 ECFA早收清單可（有）達到其預期成效
6 應該減免證所稅
7 贊成中油在觀塘興建第三天然氣接收站
8 支持中國學生納入健保
9 支持臺灣中小學（含高職、專科）服儀規定（含髮、襪、鞋）給予學生自主
10 不支持使用加密貨幣
11 不支持學雜費調漲
12 同意政府舉債發展前瞻建設計畫
13 支持電競列入體育競技
14 反對台鐵東移徵收案
15 支持陳前總統保外就醫
16 年金改革應取消或應調降軍公教月退之優存利率十八趴
17 同意動物實驗
18 油價應該凍漲或緩漲
19 反對旺旺中時併購中嘉


In [251]:
sorted(total_scores[0], key=lambda x:x[1])[::-1][:300]

[(76228, 0.8740161909270178),
 (79793, 0.8668811676031387),
 (14001, 0.8526681250214829),
 (35255, 0.8489102085647642),
 (43464, 0.8475419939546098),
 (86720, 0.8355072900014648),
 (67733, 0.8160751431158342),
 (55852, 0.8113318138358603),
 (27992, 0.8049759147242537),
 (85168, 0.8036466036167522),
 (49244, 0.8034263994684562),
 (3827, 0.8004300705349475),
 (1773, 0.7926143998391173),
 (3183, 0.7917251292384895),
 (15712, 0.7910676862360562),
 (79970, 0.783055220541361),
 (62279, 0.783055220541361),
 (57922, 0.7783713517507421),
 (32346, 0.777883622488362),
 (27993, 0.7757636334242963),
 (33356, 0.7735577431462735),
 (90405, 0.7732626533103026),
 (89859, 0.7652864926005547),
 (35313, 0.7647701412713721),
 (39351, 0.7620312978642625),
 (87286, 0.7604105326226486),
 (28760, 0.7604105326226486),
 (58854, 0.7604105326226485),
 (47956, 0.7595865603540707),
 (43725, 0.7593803497771718),
 (90348, 0.7558120777163438),
 (79797, 0.7533123497739243),
 (31976, 0.7529370591824129),
 (87433, 0.75284

In [259]:
search_result = np.zeros((20,300))
for cnt,i in enumerate(total_scores):
    keys ,values = zip(*sorted(i, key= lambda x: x[1])[::-1][:300])
    search_result[cnt] = keys

In [233]:
from sklearn.model_selection import train_test_split
batch_size = 100000
folder = 'news_data_1/'
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)
test_loader = DataLoader(test_query, batch_size=batch_size, shuffle=False)

In [234]:
class TestQuery(Dataset):
    def __init__(self, contents, quries):      
        # read data
        self.contents = contents
        self.queries  = quries
        self.indices = list(product(range(self.queries.shape[0]), range(self.contents.shape[0])))
        self.size = len(self.indices)
        
    def __getitem__(self, i):    
        j, k = self.indices[i]
        return self.queries[j], self.contents[k]
    
    def __len__(self):
        return self.size

In [235]:
from itertools import product
test_data = TestQuery(contents, test_query)

In [236]:
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [262]:
search_result=search_result.astype(np.int)

In [263]:
# results = results.reshape(20, -1)
# search_result = np.flip(np.argsort(results, axis=1), axis=1)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(300):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i]

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))
    
fname = 'simple7.csv'
df.to_csv('output/' + fname, index=False)

In [324]:
idx=search_result[1][0]
content_list[search_result[1][0]]

'有民眾於國發會的公共政策網路參與平臺提議「提升全民行車觀念，以提升全民行車環境品質，解除禁行機慢車道，解除強制二段式左轉」，交通部認為涉及地方政府道路交通工程配置規畫事宜，經函徵各地方政府意見，多數認不宜貿然廢除，應尊重地方政府的權責。\n交通部表示，因涉地方政府交通管理規畫權責，函徵全國各縣市政府、內政部警政署、公路總局及運輸研究所等單位意見，並於5月26日召開會議，邀請提案人及相關單位共同討論。中央主管機關應尊重各地道路主管機關係基於安全及效率之交通管理規劃。鑑於現行法規已留給地方政府彈性管理之空間，應尚無需研修相關法規，地方政府可依現場道路交通環境情況，因地制宜綜合評估是否設置或取消禁行機車道及兩段式左轉相關標誌標線。\n目前已有多個縣市開始試辦及追蹤實施後之事故改善情形，並逐步擴大檢討，例如臺北市在42處路段試辦取消第三車道禁行機車，結果25處事故改善，17處惡化，因沒有顯著效果，目前實作仍以個案方式檢討施行。\n交通部運輸研究所亦已就「路段共享道路空間」及「路口減少轉向衝突」之改善目標，研提具安全性及全國性可適用之規劃原則可供各縣市政府參考，該部將督同公路總局及各直轄市、縣（市）政府因地制宜檢討各級道路配置，以提供更友善的交通環境供不同族群用路人合理使用道路。\n(中時 )'

In [327]:
news_urls.values[idx]

array(['news_080023',
       'http://www.chinatimes.com/newspapers/20150107000955-260302'],
      dtype=object)

In [ ]:
import pickle 
with  open('test.pickle') as file:
    pickle.dump(, file)